# Laborator 5, tema 4: Dashboards

1. Documentati-va despre [dashboards](https://en.wikipedia.org/wiki/Dashboard_(business)) (~ "tablouri de bord"). 
1. Documentati-va despre solutiile de dashboards disponibile in jupyter notebook, de ex:
    1. [How to create a dashboard in Python with Jupyter Notebook?](https://mljar.com/blog/dashboard-python-jupyter-notebook/)
    1. [Interactive Dashboard from Jupyter Notebook with Mercury framework](https://towardsdatascience.com/interactive-dashboard-from-jupyter-notebook-with-mercury-framework-e1269fdbe73c)
    1. [How to Create a Beautiful Python Visualization Dashboard With Panel/Hvplot](https://www.youtube.com/watch?v=uhxiXOTKzfs)
    1. [Interactive data dashboards in Jupyter notebook with ipywidgets and Bokeh](https://danielmuellerkomorowska.com/2021/08/02/interactive-data-dashboards-in-jupyter-notebook-with-ipywidgets-and-bokeh/); explicatii la [How to Build an Interactive Data Dashboard in Jupyter Notebook](https://www.youtube.com/watch?v=EEKX5l_Y3_8) etc.
1. Alegeti un set de date din repository-ul [UC Irvine Machine Learning Repository](http://archive.ics.uci.edu/datasets) si realizati un dashboard pe baza lui. Prezentarea se poate face in Jupyter notebook sau ca aplicatie standalone. Includeti in arhiva:
    * fisier `requirements.txt` cu bibliotecile care trebuie instalate cu `pip`; [documentatie requirements.txt](https://learnpython.com/blog/python-requirements-file/)
    * fisier readme.txt, in limba romana sau engleza, cu pasii care trebuie urmati pentru a putea utiliza dashboard-ul; de exemplu, unul din pasi este `pip install -r requirements.txt`; adaugati alti pasi necesari. 

In [191]:
import numpy as np
import pandas as pd
import panel as pn
pn.extension('tabulator')#used for interactive data tables

import hvplot.pandas


In [192]:
df = pd.read_csv('Metro_Interstate_Traffic_Volume.csv')
#df

In [193]:
df['year'] = pd.to_datetime(df['date_time']).dt.year
df['hour'] = pd.to_datetime(df['date_time']).dt.hour
df['year'] = pd.to_numeric(df['year'])
df['hour'] = pd.to_numeric(df['hour'])
df['traffic_volume'] = pd.to_numeric(df['traffic_volume'])
#df

In [194]:
# Make DataFrame Pipeline Interactive
idf = df.interactive()

In [195]:
year_slider = pn.widgets.IntSlider(name='Year slider', start=2012, end=2018, step=1, value=2015)
year_slider

IntSlider(end=2018, name='Year slider', start=2012, value=2015)

In [196]:
x_axis = pn.widgets.RadioButtonGroup(
    name='X axis',
    options=['hour', 'weather_main'],
    button_type='success'
)
x_axis

RadioButtonGroup(button_type='success', name='X axis', options=['hour', 'weather_main'], value='hour')

In [197]:
traffic_volume_pipeline = (
    idf[
        (idf.year <= year_slider)
    ]
    .groupby([x_axis, 'year'])['traffic_volume'].mean()
    #converts to dataFrame
    .to_frame()
    #reset index so hour and weather may be regular columns
    .reset_index()
    .sort_values(by=['year', x_axis])  
    #reindexing after sorting
    .reset_index(drop=True)
)
# Display the plot
traffic_volume_pipeline

In [198]:
traffic_volume_plot = traffic_volume_pipeline.hvplot(x = 'year',
                                                     #create subplots allowing visualisation of volume
                                                     by=x_axis, 
                                                     y='traffic_volume',
                                                     line_width=2, 
                                                     title="Traffic volume by hour/weather")
traffic_volume_plot

In [214]:
traffic_volume_table = traffic_volume_pipeline.pipe(pn.widgets.Tabulator, 
                                                    #it loads data incrementally as the user navigates through pages
                                                    pagination='remote', 
                                                    page_size = 11, 
                                                    # it adjusts its width to fit the available space
                                                    sizing_mode='stretch_width') 
traffic_volume_table

In [202]:
hour_vs_traffic_volume_scatterplot_pipeline = (
    idf[
        (idf.year == year_slider) 
    ]
    .groupby(['weather_main', 'year', 'hour'])['traffic_volume'].mean()
    #converts to dataFrame
    .to_frame()
    #reset index so hour and weather may be regular columns
    .reset_index()
    .sort_values(by='year') 
    #reindexing after sorting
    .reset_index(drop=True)
)
hour_vs_traffic_volume_scatterplot_pipeline

In [203]:
hour_vs_traffic_volume_scatterplot = hour_vs_traffic_volume_scatterplot_pipeline.hvplot(x='hour', 
                                                                y='traffic_volume', 
                                                                by='weather_main', 
                                                                size=80, #pixels
                                                                kind="scatter", 
                                                                title="Traffic volume by hour based on weather",
                                                                #sets the transparency                        
                                                                alpha=0.7,
                                                                legend=True, 
                                                                height=500, 
                                                                width=500)
hour_vs_traffic_volume_scatterplot

In [204]:

traffic_volume_bar_pipeline = (
    idf[
        (idf.year == year_slider)
    ]
    .groupby(['year', x_axis])['traffic_volume'].sum()
    #converts to dataFrame
    .to_frame()
    #reset index so hour and weather may be regular columns
    .reset_index()
    .sort_values(by=x_axis)  
    #reindexing after sorting
    .reset_index(drop=True)
)
traffic_volume_bar_pipeline

In [205]:
traffic_volume_source_bar_plot = traffic_volume_bar_pipeline.hvplot(kind='bar', 
                                                     x=x_axis, 
                                                     y='traffic_volume', 
                                                     title='Traffic volume by hour/weather')
traffic_volume_source_bar_plot

In [215]:
#Layout using Template
template = pn.template.FastListTemplate(
    title='Metro Interstate Traffic Volume Dashboard', 
    sidebar=[pn.pane.Markdown("# Hourly Minneapolis-St Paul, MN traffic volume."), 
             pn.pane.Markdown("#### Hourly Interstate 94 Westbound traffic volume for MN DoT ATR station 301, roughly midway between Minneapolis and St Paul, MN."), 
             pn.pane.Markdown("## Settings"),   
             year_slider,
             pn.pane.Markdown("   "),
            x_axis],
    main=[pn.Row(pn.Column(traffic_volume_plot.panel( sizing_mode='stretch_width', height=400)), 
                 traffic_volume_table.panel( sizing_mode='stretch_width')), 
          pn.Row(hour_vs_traffic_volume_scatterplot.panel( sizing_mode='stretch_width')),
          pn.Row(traffic_volume_source_bar_plot.panel( sizing_mode='stretch_width'))],
    accent_base_color="#3481c9",
    header_background="#3481c9",
)
template.show()
template.servable();

Launching server at http://localhost:53123


Tema va fi predata pe siteul de elearning pana miercuri, 8 noiembrie 2023, ora 22. 